# Imports

In [11]:
from Bio import SeqIO
import csv
import math
import re

# Define lists

In [12]:
results = []
indexs = []
error_results =[]
error_indexs = []
sequence = []

# Read in files

In [13]:
for seq_record in SeqIO.parse("genomicJs.fasta", "fasta"):
    results.append(seq_record.description)
    ind = []
    for i in range(3):
        seq_record_temp = seq_record.seq[i:]
        floor = math.floor(len(seq_record_temp)/3)
        index = len(seq_record_temp) - (floor*3)
        if index != 0:
            seq_record_temp = seq_record_temp[:-(index)]
        # translating from dna to amino acid and find first (F/W)X(S/T)
        traslated_seq = seq_record_temp.translate()
        position = -1
        m = re.search('[FW]G.?G[ST]', str(traslated_seq))
        if m:
            position = m.start()
        index_F = ((position*3)+i)
        ind.append(index_F)
    pos_idx = [i for i in ind if i >=0]
    if len(pos_idx) != 0:
            indexs.append(str(min(pos_idx)))
    else:
            error_indexs.append(str(0))
            error_results.append(seq_record.description)
            sequence.append(seq_record.seq)


# write to file

In [14]:
with open('J_gene_CDR3_anchors_test.csv', 'w') as csv_file:
    fieldnames = ['gene','anchor_index']
    csv_writer = csv.DictWriter(csv_file,fieldnames=fieldnames,delimiter=';')
    csv_writer.writeheader()
    
    for result, index in zip(results, indexs):
        csv_writer.writerow({'gene': result, 'anchor_index': index})

# write to error file

In [15]:
with open('J_gene_CDR3_anchors_test_error', 'w') as csv_file:
        fieldnames = ['gene','sequence','anchor_index']
        csv_writer = csv.DictWriter(csv_file,fieldnames=fieldnames,delimiter=';')
        csv_writer.writeheader()

        for error_result, seq, error_index in zip(error_results, sequence, error_indexs):
            csv_writer.writerow({'gene': error_result, 'sequence':seq,'anchor_index': error_index})